In [18]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import import_ipynb
from rotation import rotation_matrix, unit_vector, angle_between, x_rotation, y_rotation, z_rotation

# for machine learning
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition
from imblearn.over_sampling import SMOTE

import csv
from csv import reader
import pickle

In [32]:
NUM_FEATURES = 3
NUM_JOINTS = 20
NUM_FRAMES = 16
FILE_NAME = '../train.csv'
test_FILE_NAME = '../test.csv'

In [24]:
dtf = pd.read_csv(FILE_NAME, header = None)
dtf = dtf.set_index(dtf.columns[0])

dtf = dtf.sample(frac = 1)
X = dtf.iloc[:,:-1]
y = dtf.iloc[:,-1:]

dtf_test = pd.read_csv(test_FILE_NAME, header = None)
dtf_test = dtf_test.set_index(dtf.columns[0])

dtf_test = dtf_test.sample(frac = 1)

In [30]:
def read_xyz(row):
    skeleton_data, label = read_skeleton(row)
    
    data = np.zeros((NUM_FRAMES, NUM_JOINTS, NUM_FEATURES))
    for m, i in enumerate(skeleton_data['frame_info']):
        for n, j in enumerate(i['joint_info']):
            feature_info = j['feature_info']
            data[m, n, :] = [feature_info['x'], feature_info['y'], feature_info['z']]

    data = data.transpose(2, 0, 1)
    return data, label

In [84]:
def read_skeleton(row):
    label = row[-1]
    #label = -1
    skeleton_data = {}
    skeleton_data['index'] = row[0]
    skeleton_data['num_frame'] = NUM_FRAMES
    skeleton_data['frame_info'] = []
    
    for frame in range(NUM_FRAMES):
        offset = NUM_JOINTS * NUM_FEATURES
        data_in_frame = row[1+frame*offset:1+(frame+1)*offset]
        frame_info = {}
        frame_info['num_joints'] = NUM_JOINTS
        frame_info['joint_info'] = []
        
        for feature in range(NUM_JOINTS):
            offset = NUM_FEATURES
            data_in_joint = data_in_frame[feature*offset:(feature+1)*offset]
            joint_info = {}
            joint_info['num_features'] = NUM_FEATURES
            joint_info['feature_info'] = {
                k: float(v)
                for k, v in zip(['x', 'y', 'z'], data_in_joint)
            }
            frame_info['joint_info'].append(joint_info)
                    
        skeleton_data['frame_info'].append(frame_info)
    return skeleton_data, label

In [6]:
def normalisation(data):
    N, C, T, V, M = data.shape
    s = np.transpose(data, [0, 4, 2, 3, 1])
    zaxis=[0, 1]
    xaxis=[8, 4]
    
    print('sub the center joint #1 (spine joint in ntu and neck joint in kinetics)')
    for i_s, skeleton in enumerate(tqdm(s)):
        if skeleton.sum() == 0:
            continue
        main_body_center = skeleton[0][:, 1:2, :].copy()
        for i_p, person in enumerate(skeleton):
            if person.sum() == 0:
                continue
            mask = (person.sum(-1) != 0).reshape(T, V, 1)
            s[i_s, i_p] = (s[i_s, i_p] - main_body_center) * mask

    print('parallel the bone between hip(jpt 0) and spine(jpt 1) of the first person to the z axis')
    for i_s, skeleton in enumerate(tqdm(s)):
        if skeleton.sum() == 0:
            continue
        joint_bottom = skeleton[0, 0, zaxis[0]]
        joint_top = skeleton[0, 0, zaxis[1]]
        axis = np.cross(joint_top - joint_bottom, [0, 0, 1])
        angle = angle_between(joint_top - joint_bottom, [0, 0, 1])
        matrix_z = rotation_matrix(axis, angle)
        for i_p, person in enumerate(skeleton):
            if person.sum() == 0:
                continue
            for i_f, frame in enumerate(person):
                if frame.sum() == 0:
                    continue
                for i_j, joint in enumerate(frame):
                    s[i_s, i_p, i_f, i_j] = np.dot(matrix_z, joint)

    print('parallel the bone between right shoulder(jpt 8) and left shoulder(jpt 4) of the first person to the x axis')
    for i_s, skeleton in enumerate(tqdm(s)):
        if skeleton.sum() == 0:
            continue
        joint_rshoulder = skeleton[0, 0, xaxis[0]]
        joint_lshoulder = skeleton[0, 0, xaxis[1]]
        axis = np.cross(joint_rshoulder - joint_lshoulder, [1, 0, 0])
        angle = angle_between(joint_rshoulder - joint_lshoulder, [1, 0, 0])
        matrix_x = rotation_matrix(axis, angle)
        for i_p, person in enumerate(skeleton):
            if person.sum() == 0:
                continue
            for i_f, frame in enumerate(person):
                if frame.sum() == 0:
                    continue
                for i_j, joint in enumerate(frame):
                    s[i_s, i_p, i_f, i_j] = np.dot(matrix_x, joint)

    data = np.transpose(s, [0, 4, 2, 3, 1])
    return data
    

In [15]:
# Stratified Train test split
sss = model_selection.StratifiedShuffleSplit(n_splits=1, test_size = 0.2, random_state=None)
for train_index, test_index in sss.split(X, y):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

scaler = preprocessing.MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#submission_test = scaler.transform(dtf_test)

In [88]:
smote = SMOTE(sampling_strategy='minority', random_state=42)

# fit predictor and target variable
x_smote, y_smote = smote.v(X_train, y_train)
x_smote.shape 

(7963, 1)

In [82]:
fp = np.zeros((len(dtf), NUM_FEATURES, NUM_FRAMES, NUM_JOINTS, 1), dtype=np.float32) #construct a matrix, with num of data, num of features for each joint, num of frames, num of joints, num of people(always 1 in our case)
for i, row in enumerate(x_smote):
    data, label = read_xyz(row)
    fp[i, :, :, :, 0] = data
np.save('train_data.npy', fp)
#np.save('test_data.npy', fp)

In [109]:
labels = []
print(就能变成array(y_smote))
for i, row in enumerate(y_smote):
    labels.append(row)
print(labels)
with open('label.pkl', 'wb') as f:
    pickle.dump(labels, f)

<class 'pandas.core.frame.DataFrame'>
[961]


In [61]:
fp = np.zeros((len(dtf), NUM_FEATURES, NUM_FRAMES, NUM_JOINTS, 1), dtype=np.float32)
    #construct a matrix, with num of data, num of features for each joint, num of frames, num of joints, num of people(always 1 in our case)
with open(FILE_NAME, 'r') as f:
        csv_reader = reader(f)
        labels = []
        for i, row in enumerate(csv_reader):
            data, label = read_xyz(row)
            labels.append(int(label))
            fp[i, :, :, :, 0] = data
#fp = normalisation(fp)
np.save('train_data.npy', fp)
#np.save('test_data.npy', fp)

# with open('label.pkl', 'wb') as f:
#     pickle.dump(labels, f)

962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961


962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961


962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961


962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961


962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961


962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961


962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961


962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961


962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961
962
961


In [99]:
unpickled_df = pd.read_pickle("label.pkl")
unpickled_df

[961]